In [0]:
import numpy as np
import pandas as pd
import math
import io


In [2]:
from google.colab import files
uploaded = files.upload()

Saving projdata.csv to projdata.csv


In [4]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "projdata.csv" with length 132145 bytes


In [0]:




df=pd.read_csv(io.StringIO(uploaded['projdata.csv'].decode("utf-8")),engine='python',sep=",")


In [0]:
y=np.array(df[['Fault']])
X=df.as_matrix(columns=df.columns[1:])
X=X.T
y=y.T

In [0]:
def sig(x):
    return 1/(1+np.exp(-x))

In [0]:
def relu(x):
    return x*(x>0)

In [0]:
def leaky_relu(x):
    y=x*(x>0)
    for i in range(len(np.max(y.shape))):
        if y[i]==0:
            y[i]==0.01*x[i]

In [10]:
nx=np.min(np.shape(X))
ny=np.min(np.shape(y))
nh=3
nl=2
print("Size of input layer: "+str(nx))
print("Size of output layer: "+str(ny))
print("Size of hidden units in each layer: "+str(nh))
print("No of hidden layers: "+str(nl))


Size of input layer: 3
Size of output layer: 1
Size of hidden units in each layer: 3
No of hidden layers: 2


In [0]:
def initialise_params(layer_dims):
    
    L=len(layer_dims)
    
    params={}
    
    for l in range(1,L,1):
        params["W"+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])*np.sqrt(1/layer_dims[l-1])
        params["b"+str(l)]=np.zeros((layer_dims[l],1))
            
    for l in range(1,L):
        assert (params["W"+str(l)].shape == (layer_dims[l],layer_dims[l-1]))
        assert (params["W"+str(l)].shape == (layer_dims[l],layer_dims[l-1]))
        
    
    return params
    

In [0]:

params=initialise_params([nx,nh,nh,ny])

In [0]:
def forward_prop(X,params):
    
    W1=params["W1"]
    b1=params["b1"]
    W2=params["W2"]
    b2=params["b2"]
    W3=params["W3"]
    b3=params["b3"]
    
    Z1=np.dot(W1,X)+b1
    A1=np.tanh(Z1)
    Z2=np.dot(W2,A1)+b2
    A2=np.tanh(Z2)
    Z3=np.dot(W3,A2)+b3
    A3=sig(Z3)
    
    cache={"Z1":Z1,"A1":A1,"Z2":Z2,"A2":A2,"Z3":Z3,"A3":A3}
    
    return A3,cache

In [0]:
A3,cache=forward_prop(X,params)

In [0]:
def compute_cost(y,A3,params,lambd):
    
    m=np.max(y.shape)
        
    c1=np.dot(y,(np.log(A3)).T)
    c2=np.dot((1-y),(np.log(1-A3)).T)
    
    w1=np.sum(np.square(params["W1"]))
    w2=np.sum(np.square(params["W2"]))
    w3=np.sum(np.square(params["W3"]))
    
    c3=(lambd/(2*m))*(w1+w2+w3)
    
    cost = -((c1+c2)/m)+c3
    
    cost = np.squeeze(cost) 
    return (np.sum(cost))

In [17]:
cost=compute_cost(y,A3,params,lambd=0.7)
print(cost)

0.7095100595789434


In [0]:
def back_prop(y,params,cache,X,lambd):
    
    W1=params["W1"]
    b1=params["b1"]
    W2=params["W2"]
    b2=params["b2"]
    W3=params["W3"]
    b3=params["b3"]
    
    m=np.max(y.shape)
    
    A1=cache["A1"]
    A2=cache["A2"]
    A3=cache["A3"]
    
    
    dZ3 = A3-y
    dW3 = np.dot(dZ3,A3.T)/m+((lambd/m)*W3)
    db3 = np.sum(dZ3,axis=1,keepdims=True)/m
    dZ2 = np.dot(W3.T,dZ3)*(1-np.power(A2,2))
    dW2 = np.dot(dZ2,A2.T)/m+((lambd/m)*W2)
    db2 = np.sum(dZ2,axis=1,keepdims=True)/m
    dZ1 = np.dot(W2.T,dZ2)*(1-np.power(A1,2))
    dW1 = np.dot(dZ1,X.T)/m+((lambd/m)*W1)
    db1 = np.sum(dZ1,axis=1,keepdims=True)/m
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2,
             "dW3": dW3,
             "db3": db3}
    
    return grads


In [0]:
grads=back_prop(y,params,cache,X,lambd=0.7)

In [0]:
def update_params(grads,params,learning_rate):
    
    W1 = params["W1"]
    b1 = params["b1"]
    W2 = params["W2"]
    b2 = params["b2"]
    W3 = params["W3"]
    b3 = params["b3"]
    
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    dW3 = grads["dW3"]
    db3 = grads["db3"]
    
    
    W1 = W1-(learning_rate*dW1)
    b1 = b1-(learning_rate*db1)
    W2 = W2-(learning_rate*dW2)
    b2 = b2-(learning_rate*db2)
    W3 = W3-(learning_rate*dW3)
    b3 = b3-(learning_rate*db3)
    
    
    params = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,"W3":W3,"b3":b3
                }
    
    return params
    


In [0]:
params=update_params(grads,params,learning_rate=0.01)

In [0]:
def nn_model(X,y,nh,num_of_iters,print_cost=False):
    
    nx=np.min(X.shape)
    ny=np.min(y.shape)
    
    parameters=initialise_params([nx,nh,nh,ny])
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    learning_rate=0.005
    lambd=0.2
    
    for i in range(0,num_of_iters,1):
        
        A3,cache=forward_prop(X,parameters)
        
        cost=compute_cost(y,A3,parameters,lambd)
        
        grads=back_prop(y,parameters,cache,X,lambd)
        
        parameters=update_params(grads,parameters,learning_rate)
        
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        
    return parameters

In [23]:
params=nn_model(X,y,nh,num_of_iters=50000,print_cost=True)

Cost after iteration 0: 0.706006
Cost after iteration 1000: 0.658892
Cost after iteration 2000: 0.627603
Cost after iteration 3000: 0.543897
Cost after iteration 4000: 0.503840
Cost after iteration 5000: 0.470877
Cost after iteration 6000: 0.438751
Cost after iteration 7000: 0.411505
Cost after iteration 8000: 0.384904
Cost after iteration 9000: 0.334921
Cost after iteration 10000: 0.239346
Cost after iteration 11000: 0.205367
Cost after iteration 12000: 0.173413
Cost after iteration 13000: 0.149311
Cost after iteration 14000: 0.130839
Cost after iteration 15000: 0.110845
Cost after iteration 16000: 0.028748
Cost after iteration 17000: 0.026552
Cost after iteration 18000: 0.025126
Cost after iteration 19000: 0.023999
Cost after iteration 20000: 0.023046
Cost after iteration 21000: 0.022218
Cost after iteration 22000: 0.021488
Cost after iteration 23000: 0.020839
Cost after iteration 24000: 0.020256
Cost after iteration 25000: 0.019730
Cost after iteration 26000: 0.019250
Cost after ite

In [0]:
def predict(X,params):
    
    A3, cache = forward_prop(X,params)
    predictions = (A3>0.5)
    
    return predictions


In [25]:
predictions = predict(X,params)
print("predictions mean = " + str(np.mean(predictions)))

print ('Accuracy: %d' % float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100) + '%')

predictions mean = 0.695771219957408
Accuracy: 100%


In [26]:
hidden_layer_sizes = [3, 4, 5, 15, 16, 17, 18, 19, 20]
for i, nh in enumerate(hidden_layer_sizes):
    parameters = nn_model(X, y, nh, num_of_iters = 15000)
    predictions = predict(X,parameters)
    accuracy = float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100)
    print ("Accuracy for {} hidden units: {} %".format(nh, accuracy))

Accuracy for 3 hidden units: 72.13264374809857 %
Accuracy for 4 hidden units: 66.41314268329785 %
Accuracy for 5 hidden units: 82.84149680559781 %
Accuracy for 15 hidden units: 96.37967751749315 %
Accuracy for 16 hidden units: 99.90873136598722 %
Accuracy for 17 hidden units: 100.0 %
Accuracy for 18 hidden units: 100.0 %
Accuracy for 19 hidden units: 100.0 %
Accuracy for 20 hidden units: 100.0 %
